<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/Scraping/1_HTTP_Inicial/ejercicio/7Puentes_ejercicios_web_scraping_http_inicial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" data-canonical-src="https://colab.research.google.com/assets/colab-badge.svg"></a>

# **Web scraping con Python**

## Más ejercicios!
<img src="https://c.tenor.com/S4Sz_yvlLn4AAAAC/cats-cat.gif" height=250>


## **Vamos a scrapear las novedades de Goodreads.com**
<hr>
<img src='https://digital.hbs.edu/platform-digit/wp-content/uploads/sites/2/2015/10/goodreads-620x200.png'>

- GoodReads es una red social donde se comparte lo que uno esta leyendo, se ofecen recomendaciones de lectura, etc.

- En particular nos ofrece una serie de titulos catalogados como "novedades" para varios generos entrando a: https://www.goodreads.com/genres/


El objetivo del ejercicios será *scrapear* esas novedades para un género de nuestro interés (o para todos!)

1. Obtener el HTML de https://www.goodreads.com/genres/new_releases/{CATEGORIA}, siendo CATEGORIA el género de interés
2. Entrar a cada uno de los links de los libros
3. Obtener el título y el resumen
4. Guardar toda la informacion en un archivo *.csv* 

In [22]:
# Importamos librerias
from bs4 import BeautifulSoup
import requests
import json
import time
import re 
import pandas as pd

In [50]:
url = 'https://www.goodreads.com/genres/new_releases/economics'

In [63]:
headers = {
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
        }

In [64]:
html_obtained = requests.get(url, headers=headers)

In [89]:
html_obtained

<Response [200]>

In [90]:
soup = BeautifulSoup(html_obtained.text, "html.parser")

In [106]:
books_container = soup.find_all('div', class_='coverWrapper')

In [111]:
books_container

[<div class="coverWrapper" id="bookCover714883_52979663">
 <a href="/book/show/52979663-change-everything"><img alt="Change Everything: Racial Capitalism and the Case for Abolition" class="bookImage" src="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1624138029l/52979663._SX318_.jpg" title="" width="115"/></a>
 </div>,
 <div class="coverWrapper" id="bookCover714883_55529461">
 <a href="/book/show/55529461-breaking-things-at-work"><img alt="Breaking Things at Work: The Luddites Are Right About Why You Hate Your Job" class="bookImage" src="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1601570768l/55529461.jpg" title="" width="115"/></a>
 </div>]

In [112]:
for book in books_container:
    print(book['a'])

KeyError: 'a'

In [98]:
book_name = [book.get('alt') for book in books_container]

In [97]:
book_name

[]

In [67]:
soup

<!DOCTYPE html>

<html class="desktop withSiteHeaderTopFullImage">
<head>
<title>Economics Books</title>
<meta content="Economics genre: new releases and popular books, including Change Everything: Racial Capitalism and the Case for Abolition by Ruth Wilson Gilmore und Bre..." name="description"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://www.goodreads.com/genres/economics" rel="canonical"/>
<meta content="noindex" name="robots"/>
<!-- * Copied from https://info.analytics.a2z.com/#/docs/data_collection/csa/onboard */ -->
<script>
  //<![CDATA[
    !function(){function n(n,t){var r=i(n);return t&&(r=r("instance",t)),r}var r=[],c=0,i=function(t){return function(){var n=c++;return r.push([t,[].slice.call(arguments,0),n,{time:Date.now()}]),i(n)}};n._s=r,this.csa=n}();
    
    if (window.csa) {
      window.csa("Config", {
        "Application": "GoodreadsMonolith",
        "Events.SushiEndpoint": "https://unagi.amazon.com/1/events/com.amazon.csm.csa.prod",

Documentacion de nuestra función: 
 
  Scraper de libros de resumenes de libros que aparecen como 'New realeases' en GoodReads
 
  parameters:
            url_category  - String conteniendo una URL del tipo ' https://www.goodreads.com/genres/new_releases/{CATEGORIA} ' 
            base_url - String conteniendo la URL base de Goodreads. Actualmente: 'https://www.goodreads.com'
 
  return:
            titulos       - Lista de strings con los titulos de los libros correspondientes a la categoria 
            resumenes     - Lista de strings con los resumenes de los libros correspondientes a la categoria 
            df            - Pandas Dataframe cuyos índices son los títulos de los libros y cuya única columna contiene los resúmenes
 
   

URL de categoria: https://www.goodreads.com/genres/new_releases/Biography 
Status code: 200
URL de libro: https://www.goodreads.com/book/show/58558175-ancestor-trouble 
Status code: 200
El titulo es: 'Ancestor Trouble: A Reckoning and a Reconciliation

,Resumenes
Ancestor Trouble: A Reckoning and a Reconciliation,An acclaimed writer goes searching for the tru...
"The Other Dr. Gilmer: Two Men, a Murder, and an Unlikely Fight for Justice",A rural physician learns that a former doctor ...
Comedy Comedy Comedy Drama,"In this hilarious, heartfelt memoir, the star ..."
In Love: A Memoir of Love and Loss,This powerful memoir by New York Times bestsel...
No hay titulo,Multi-awardwinning Hannah Gadsby transformed c...
Ten Steps to Nanette,"From Helen Rappaport, the New York Times bests..."
After the Romanovs: Russian Exiles in Paris from the Belle Époque Through Revolution and War,"A beautifully written, witty memoir that is al..."
"Every Good Boy Does Fine: A Love Story, in Music Lessons",A landmark exploration of one of the most cons...
The Invisible Kingdom: Reimagining Chronic Illness,AN INSTANT NATIONAL BESTSELLERMemoir meets cra...
Body Work: The Radical Power of Personal Narrative,A poignant and hilarious memoir from the cultu...


 
 Resumen de 'Ancestor Trouble: A Reckoning and a Reconciliation':  



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Si se coparon mucho pueden hacer un mega scrapeo iterando sobre algunas de las categorias
 
### **Tip para que no les raje colab:**
* En la consola del navegador pegan esto y le dan enter:



```javascript

// Código en JavaScript
function ClickConnect() {
  console.log('Trabajando...')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}
intervalTiming = setInterval(ClickConnect, 60000)
```
 
Para desactivarlo: 
 
```javascript
clearInterval(intervalTiming)
```
 
 
 
 
 
<img src='https://bookbub-res.cloudinary.com/image/upload/f_auto,q_auto/v1584035706/blog/justification-for-too-many-books-memes-we-need-more-books.jpg'>

Art
URL de categoria: https://www.goodreads.com/genres/new_releases/Art 
Status code: 200
URL de libro: https://www.goodreads.com/book/show/58275730-woman-on-fire 
Status code: 200
El titulo es: 'Woman on Fire' 

Resumen: From the author of the award-winning Fugitive Colors and The Unbreakables, a gripping tale of a young, ambitious journalist embroiled in an international art art scandal centered around a Nazi-looted masterpiece--forcing the ultimate showdown between passion and possession, lovers and liars, history and truth. NOW A NEW YORK TIMES BESTSELLER! Actress Sharon Stone set to produce and star in the film adaptation of WOMAN ON FIRE. After talking her way into a job with Dan Mansfield, the leading investigative reporter in Chicago, rising young journalist Jules Roth is given an unusual--and very secret--assignment. Dan needs her to locate a painting stolen by the Nazis more than 75 years earlier: legendary Expressionist artist Ernst Engel's most famous work, Woman on Fire. W

In [ ]:
diccionario_de_dataframes['Biography']

,Resumenes
Ancestor Trouble: A Reckoning and a Reconciliation,An acclaimed writer goes searching for the tru...
No hay titulo,An Indigenous artist blends the aesthetics of ...
Comedy Comedy Comedy Drama,This powerful memoir by New York Times bestsel...
In Love: A Memoir of Love and Loss,Liz Scheier’s darkly funny and touching memoir...
Never Simple,Multi-awardwinning Hannah Gadsby transformed c...
Ten Steps to Nanette,"From Helen Rappaport, the New York Times bests..."
After the Romanovs: Russian Exiles in Paris from the Belle Époque Through Revolution and War,"A beautifully written, witty memoir that is al..."
"Every Good Boy Does Fine: A Love Story, in Music Lessons",A landmark exploration of one of the most cons...
The Invisible Kingdom: Reimagining Chronic Illness,AN INSTANT NATIONAL BESTSELLERMemoir meets cra...
Body Work: The Radical Power of Personal Narrative,All Kait Leddy had ever wanted was a little si...
